In [1]:
#pip install mysqlclient

In [138]:
import json
from Bio import Entrez
from Bio import Medline

import os

# pubmed

In [139]:
Entrez.email = "tariqf549@gmail.com"
handle = Entrez.einfo() # or esearch, efetch, ...
record = Entrez.read(handle)
handle.close()

In [140]:
querylist = ('clinical trial[Title/Abstract]' , 'italy[Title/Abstract]' , 'netherlands[Title/Abstract]' , 'case control study' , 'epidemiology' , 
             'mortality', '(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])' )

In [141]:
#myterm = MainTerm + """ AND trial[Title/Abstract] """

In [142]:
resultsfilename='results.json'

In [143]:
jsonfilename = resultsfilename
if os.path.isfile(jsonfilename):
    mainDict = json.load(open(jsonfilename))
else:
    mainDict = {}


In [144]:
def add2dict(dataresults , Q):
    for hit in dataresults:
        m1 = 'PMID' + hit
        parse_res = Medline.read(m1.split('\n'))
        
        PMID = parse_res['PMID']
        Tag = Q
        if PMID in mainDict.keys():
            print('PMID exists')
            if Q not in mainDict[PMID]['Tag']:
                 mainDict[PMID]['Tag'] = mainDict[PMID]['Tag']+';'+Tag
            continue
        else:
            Title = parse_res['TI']
            dateP = parse_res['DP']
            dateC = parse_res['EDAT']
            
            if 'JT' in parse_res.keys():
                JournalName  = parse_res['JT']
            else: JournalName = ''

            if 'LR' in parse_res.keys():
                dateMod  = parse_res['LR']
            else: dateMod = ''

            if 'AB' in parse_res.keys():
                Abstract = parse_res['AB']
                tempAbs = Abstract.split(' ')
                lastN=0
                newabs = []
                for N in range(30, len(tempAbs)+30, 30) :
                    newabs.append(' '.join(tempAbs[lastN:N]))
                    lastN= N
                Abstract = '\n'.join(newabs)
                
                
            else: Abstract = 'NA'
            Link= 'https://www.ncbi.nlm.nih.gov/pubmed/{}'.format(PMID)   


            mainDict[PMID] = {'PMID': PMID, 'Title':Title ,
                            'JournalName':JournalName ,
                              'Creation Date':dateC ,
                            'Modification Date':dateMod , 
                            'Publication Date':dateP , 
                            'Abstract':Abstract , 
                            'Link':Link,
                            'Tag':Tag   }
            print('Added new PMID: {}'.format(PMID))


In [145]:
def search_function (MyTerms, startD , endD):

    Entrez.email = "tariqf549@gmail.com"
    MainTerm = """(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] ))))"""
    
    #MainTerm = '"COVID-19"[All Fields]'
    DateRange = '"{}"[PDat] : "{}"[PDat]'.format(startD , endD)
    myterm = MainTerm + ' AND ' + MyTerms + ' AND ' + DateRange
    print(myterm)
    search_results = Entrez.read(
        Entrez.esearch(
            db="pubmed", term=myterm,  datetype="pdat", usehistory="y" , sort = 'relevance' 
        )
    )
    count = int(search_results["Count"])
    print("Found %i results" % count)

    batch_size = 10
    out_handle = open("corona_{}_papers.txt".format(MyTerms.split('[')[0]), "w")
    for start in range(0, count, batch_size):
        end = min(count, start + batch_size)
        print("Going to download record %i to %i" % (start + 1, end))
        fetch_handle = Entrez.efetch(
            db="pubmed",
            rettype="medline",
            retmode="text",
            retstart=start,
            retmax=batch_size,
            webenv=search_results["WebEnv"],
            query_key=search_results["QueryKey"],
        )
        data = fetch_handle.read()
        
        dataresults = data.split('\nPMID')[1:]
        add2dict(dataresults , MyTerms)
        
        fetch_handle.close()
        out_handle.write(data)
    out_handle.close()

In [146]:

#search_function(querylist[3] , '2020/01/01' , '2020/03/27' )

In [147]:
#mainDict['32107910']

In [148]:
for X in querylist:
    search_function(X , '2020/01/01' , '2020/03/27' )

(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND clinical trial[Title/Abstract] AND "2020/01/01"[PDat] : "2020/03/27"[PDat]
Found 5 results
Going to download record 1 to 5
Added new PMID: 32081636
Added new PMID: 32149769
Added new PMID: 32164424
Added new PMID: 32108352
Added new PMID: 32054787
(((("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields]) AND ("COVID-19"[All Fields] OR "severe acute respiratory syndrome coronavirus 2"[Supplementary Concept] OR "severe acute respiratory syndrome coronavirus 2"[All Fields] OR "2019-nCoV"[All Fields] OR "SARS-CoV-2"[All Fields] OR "2019nCoV"[All Fields] )))) AND italy[Title/Abstract] AND "2020/01/01"[PDat] : "2020/03/27"[PDat]
Found 18 results
Going to download record 1 to 1

PMID exists
PMID exists
PMID exists
Added new PMID: 32097725
PMID exists
Added new PMID: 32096367
Added new PMID: 32096116
Added new PMID: 32093921
PMID exists
Added new PMID: 32092539
Going to download record 191 to 200
Added new PMID: 32092296
Added new PMID: 32091395
PMID exists
Added new PMID: 32088333
PMID exists
PMID exists
Added new PMID: 32087122
Added new PMID: 32086938
Added new PMID: 32086236
Added new PMID: 32085843
Going to download record 201 to 210
Added new PMID: 32085842
Added new PMID: 32085839
Added new PMID: 32083985
Added new PMID: 32083728
PMID exists
PMID exists
Added new PMID: 32080723
Added new PMID: 32079150
Added new PMID: 32078596
Added new PMID: 32078595
Going to download record 211 to 220
Added new PMID: 32078069
Added new PMID: 32077441
Added new PMID: 32077440
PMID exists
PMID exists
Added new PMID: 32072794
Added new PMID: 32072569
Added new PMID: 32070466
PMID exists
Added new PMID: 32070391
Going to download record 221 to 230
Added new PMID: 32066541


Added new PMID: 32180175
Added new PMID: 32180140
Added new PMID: 32179908
Added new PMID: 32179860
PMID exists
Added new PMID: 32179125
Added new PMID: 32179124
PMID exists
PMID exists
Added new PMID: 32178716
Going to download record 121 to 130
Added new PMID: 32178711
PMID exists
PMID exists
PMID exists
PMID exists
Added new PMID: 32175421
Added new PMID: 32174129
Added new PMID: 32174128
Added new PMID: 32174095
PMID exists
Going to download record 131 to 140
Added new PMID: 32173725
Added new PMID: 32173576
Added new PMID: 32173574
PMID exists
PMID exists
Added new PMID: 32173110
PMID exists
PMID exists
Added new PMID: 32172546
PMID exists
Going to download record 141 to 150
Added new PMID: 32172226
Added new PMID: 32171948
Added new PMID: 32171872
PMID exists
PMID exists
Added new PMID: 32171866
PMID exists
PMID exists
PMID exists
PMID exists
Going to download record 151 to 160
Added new PMID: 32171059
PMID exists
Added new PMID: 32169481
PMID exists
Added new PMID: 32168162
PMID

In [151]:
with open(resultsfilename , 'w') as fp:   
    Output=[]
    for Key,Item in mainDict.items():
        print(Item['Tag'])
        Output.append(Item)
    json.dump(Output, fp)

clinical trial[Title/Abstract];epidemiology;(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];mortality;(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
clinical trial[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
italy[Title/Abstract];epidemiology
italy[Title/Abstract];epidemiology;(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
italy[Title/Abstract];epidemiology;mortality
italy[Title/Abstract];(treatment[All Fields] OR drug[All Fields] OR intervention[All Fields] OR recovery[All Fields])
italy[Title/A

In [136]:
headerslist = []
for H in ('PMID',
          'Title',
          'Link',
            'JournalName',
          'Creation Date',
            'Modification Date',
            'Publication Date',
            'Abstract',
            'Tag'):
    print(H)
    headers = '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/{}", "noInherit,noTruncate")'.format(H)
    headerslist.append(headers)
    
headerslist

PMID
Title
Link
JournalName
Creation Date
Modification Date
Publication Date
Abstract
Tag


['=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/PMID", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Title", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Link", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/JournalName", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Creation Date", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Modification Date", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/results.json", "/Publication Date", "noInherit,noTruncate")',
 '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coro

with open('ct.json' , 'w') as fp:
    json.dump(mainDict['clinical trial[Title/Abstract]'], fp)
    

In [ ]:
#pip install MySQL-python

In [10]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="yourusername",
  passwd="yourpassword"
)
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE mydatabase")
print(mydb)

ModuleNotFoundError: No module named 'mysql'

In [ ]:
print("\033[44;33mHello World!\033[m")


# google

In [ ]:
import requests
from bs4 import BeautifulSoup

query = '"covid-19"  italy'
url =  'https://scholar.google.com/scholar?start=0&q='+ query + '&hl=en&scisbd=1&as_sdt=1,5&as_vis=1&ie=UTF-8&oe=UTF-8&hl=en&btnG=Search'

content = requests.get(url).text
page = BeautifulSoup(content, 'html')
results = []
for entry in page.find_all("h3", attrs={"class": "gs_rt"}):
    results.append({"title": entry.a.text, "url": entry.a['href']})

In [ ]:
for entry in page.find_all(attrs={"class": "gs_rs"}):
    maintext = entry.get_text()
    D = maintext.split(' ')[0]
    print(D)
    print(entry.get_text())
    print()

In [ ]:
results[0]